<a href="https://colab.research.google.com/github/Harshkotkar/AI-agents-/blob/main/chatbot_with_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install lxml_html_clean
!pip install pinecone
!pip install datetime

In [20]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.7/167.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=23e592e868189671bf2b3851186cbedd3a2bb5c4969285ac70d23bc06ba702e3
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [5]:
!pip install -U langchain langchain_google_genai langchain_core langchain_community langchain_chroma langchain_huggingface

In [6]:
!pip install bs4
!pip install duckduckgo-search
!pip install  feedparser requests newspaper3k
!pip install duckduckgo-search
!pip install re
!pip install ddgs


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [7]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import bs4
from langchain_core.prompts import ChatPromptTemplate
import re
from ddgs import DDGS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
import json
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
import os
import getpass
os.environ["google_api_key"]=getpass.getpass("enter the google api key ")

enter the google api key ··········


In [9]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",temperature=0)


In [10]:
memory=ConversationBufferMemory(history=True)

In [11]:
RSS_FEEDS = {
    "economics": [
        "https://www.ft.com/rss/home",
        "https://economictimes.indiatimes.com/rssfeedsdefault.cms",
        "https://www.cnbc.com/id/10001147/device/rss/rss.html"
    ],
    "health": [
        "https://rss.medicalnewstoday.com/featurednews.xml",
        "http://rss.cnn.com/rss/cnn_health.rss",
        "https://www.who.int/feeds/entity/mediacentre/news/en/rss.xml"
    ],
    "politics": [
        "https://www.politico.com/rss/politics.xml",
        "https://thehill.com/rss/syndicator/19110",
        "http://feeds.reuters.com/Reuters/PoliticsNews"
    ],
    "technology": [
        "http://feeds.feedburner.com/TechCrunch/",
        "https://www.theverge.com/rss/index.xml",
        "https://www.wired.com/feed/rss"
    ],
    "sports": [
        "http://www.espn.com/espn/rss/news",
        "http://feeds.bbci.co.uk/sport/rss.xml?edition=uk",
        "https://bleacherreport.com/articles/feed"
    ]
}


In [12]:
# import feedparser

# def fetch_feed(feed_url):
#     feed = feedparser.parse(feed_url)
#     articles = []
#     for entry in feed.entries[:10]:  # limit to 10 latest
#         articles.append({
#             "title": entry.title,
#             "link": entry.link,
#             "published": entry.get("published", ""),
#             "summary": entry.get("summary", "")
#         })
#     return articles

# def get_domain_news(domain):
#     feeds = RSS_FEEDS.get(domain, [])
#     all_articles = []
#     for url in feeds:
#         all_articles.extend(fetch_feed(url))
#     return all_articles


In [13]:
# def fetch_feed(feed_url):
#     feed = feedparser.parse(feed_url)
#     articles = []
#     for entry in feed.entries[:10]:
#         # Fetch the full article HTML
#         response = requests.get(entry.link, timeout=10)
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Try to extract main content (depends on site structure)
#         # Many sites use <article> or <div class="content"> etc.
#         article_text = ""
#         if soup.find("article"):
#             article_text = " ".join([p.get_text() for p in soup.find("article").find_all("p")])
#         else:
#             article_text = " ".join([p.get_text() for p in soup.find_all("p")])

#         articles.append({
#             "title": entry.title,
#             "link": entry.link,
#             "published": entry.get("published", ""),
#             "content": article_text.strip()
#         })
#     return articles


In [14]:
import feedparser
from newspaper import Article

def fetch_full_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return {
        "title": article.title,
        "authors": article.authors,
        "published": article.publish_date,
        "text": article.text
    }

def fetch_feed(feed_url):
    feed = feedparser.parse(feed_url)
    articles = []
    for entry in feed.entries[:10]:  # limit to 5 latest
        try:
            full_article = fetch_full_article(entry.link)
            articles.append({
                "title": full_article["title"],
                "link": entry.link,
                "published": full_article["published"],
                "content": full_article["text"]
            })
        except Exception as e:
            print(f"Error fetching {entry.link}: {e}")
    return articles


In [15]:


def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # collapse whitespace
    text = text.strip()
    return text

def normalize_article(article, category=None):
    """
    article = {
        "title": "",
        "link": "",
        "published": "",
        "content": ""
    }
    """

    cleaned = clean_text(article["content"])

    metadata = {
        "title": article["title"],
        "url": article["link"],
        "published": str(article["published"]),
        "category": category
    }

    return {"content": cleaned, "metadata": metadata}


In [16]:

def web_search(query: str) -> str:
  # web search using DuckDuckGo (ddgs) and return top results.
    try:
        results = []
        with DDGS() as ddgs:
            for r in ddgs.text(query, region='wt-wt', safesearch='moderate', timelimit='m', max_results=6):
                title = r.get("title", "")
                body = r.get("body", "")
                link = r.get("href", "")
                results.append(f"Title: {title}\nSnippet: {body}\nURL: {link}")
        return "\n\n".join(results) if results else "No results found."
    except Exception as e:
        return f"Search failed: {e}"


In [17]:
print(web_search("navle bridge accident"))


Title: NAVLE - ICVA
Snippet: NAVLE candidates who graduated from a school or college of veterinary medicine not listed as an AVMA-accredited veterinary school must obtain NAVLE eligibility by completing a certification …
URL: https://www.icva.net/navle/

Title: Sample Questions-NAVLE | ICVA
Snippet: These sample questions were chosen to represent the various species and content areas of the examination, but they do not address all of the NAVLE content areas as defined in the blueprint.
URL: https://www.icva.net/navle/sample-questions-navle/

Title: 2024-2025 NAVLE® - ICVA
Snippet: The NAVLE consists of 360 clinically relevant multiple-choice questions and is available during predetermined testing windows in November–December 2024 and April 2025. This NAVLE …
URL: https://www.icva.net/image/cache/2024-2025_BOI_Final.pdf

Title: ICVA NAVLE® Dashboard
Snippet: Welcome to the NAVLE Candidate Dashboard! The dashboard contains status details related to the current NAVLE testing window and an

In [18]:
model_name="BAAI/bge-Base-en-v1.5"
model=HuggingFaceEmbeddings(model_name=model_name,
                                model_kwargs={"device":"cpu"},
                                encode_kwargs={"normalize_embeddings":True})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [40]:

import datetime
def normalize_document(
    title: str,
    content: str,
    url: str,
    published=None,
    category=None,
    source="unknown"
):
    cleaned_content = clean_text(content)

    # Convert published to ISO format if possible
    if isinstance(published, datetime.datetime):
        published = published.isoformat()
    else:
        try:
            published = str(published) if published else None
        except:
            published = None

    metadata = {
        "title": title or "",
        "url": url or "",
        "published": published,
        "category": category,
        "source": source
    }

    return {
        "content": cleaned_content,
        "metadata": metadata
    }


In [27]:
from newspaper import Article

def extract_metadata(url):
    article = Article(url)
    article.download()
    article.parse()

    title = article.title
    published = article.publish_date

    return title, published


In [30]:
# import requests
# from unstructured.partition.html import partition_html

# def extract_with_unstructured(url):
#     html = requests.get(url).text
#     elements = partition_html(text=html)

#     # Join only meaningful text
#     full_text = "\n".join([e.text for e in elements if hasattr(e, 'text')])

#     return full_text
import requests
from unstructured.partition.html import partition_html

def extract_with_unstructured(url):
    html = requests.get(url).text
    elements = partition_html(text=html)

    # Join meaningful text only
    clean_text = "\n".join([e.text for e in elements if hasattr(e, 'text')])
    return clean_text


In [32]:
# normalized = normalize_document(
#     title=title_from_scraper,
#     content=full_text,
#     url=url,
#     published=publish_date,
#     category=category,
#     source="scraper_unstructured"
# )


In [51]:
url = "https://edition.cnn.com/2025/11/16/uk/britain-asylum-reform-shabana-mahmood-hnk-intl"   # ← test URL

# Step A: Get text content from Unstructured
full_text = extract_with_unstructured(url)

# Step B: Get metadata from Newspaper
title_from_scraper, publish_date = extract_metadata(url)

# Step C: Normalize document
normalized = normalize_document(
    title=title_from_scraper,
    content=full_text,
    url=url,
    published=publish_date,

    source="scraper_unstructured"
)


In [52]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)


In [53]:
def chunk_document(normalized_doc):
    content = normalized_doc["content"]
    metadata = normalized_doc["metadata"]

    chunks = text_splitter.split_text(content)

    # attach metadata to each chunk
    chunked_docs = [
        {
            "content": chunk,
            "metadata": metadata
        }
        for chunk in chunks
    ]

    return chunked_docs


In [54]:
chunked_docs = chunk_document(normalized)


In [55]:
chunked_docs

[{'content': "UK Home Secretary Shabana Mahmood arrives at 10 Downing Street to attend the nation's weekly cabinet meeting in London in October 2025. See all topics Email Link Copied! Britain has announced plans to carry out the most significant reform of its asylum seeker policies in years, overhauling immigration rules in response to surging anti-immigrant sentiment and challenges from right-wing parties. The new policy, which will be unveiled by the ruling Labour government in full on Monday, is partially modeled on Denmark’s approach, which has one of Europe’s toughest rules for refugees. Labour has been hardening its immigration policies, particularly on illegal small-boat crossings from France, to counter the rising populist Reform UK party which advocates for “freezing immigration” and “stopping the boats",
  'metadata': {'title': 'Britain’s Shabana Mahmood to announce ‘mist significant’ changes to country’s asylum rules in years',
   'url': 'https://edition.cnn.com/2025/11/16/u

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)
